In [1]:
import pandas as pd
from ydata_profiling import ProfileReport

In [2]:
colunas_geral = [
    # Localização
    'SG_UF_PROVA',
    
    # Presença
    'TP_PRESENCA_CN', 
    'TP_PRESENCA_CH', 
    'TP_PRESENCA_LC', 
    'TP_PRESENCA_MT',
    'TP_PRESENCA_GERAL',
    'TP_PRESENCA_REDACAO',
    
    # Notas
    'NU_NOTA_CN', 
    'NU_NOTA_CH', 
    'NU_NOTA_LC', 
    'NU_NOTA_MT',
    'NU_NOTA_REDACAO'
]

colunas_aspectos_sociais = [
    # Localização
    'SG_UF_PROVA',
    
    # Variáveis sociais
    'TP_SEXO',
    'TP_COR_RACA',
    'TP_ESTADO_CIVIL',
    'TP_FAIXA_ETARIA',
    'TP_ST_CONCLUSAO',
    'TP_DEPENDENCIA_ADM_ESC',
    'TP_ESCOLA',
    'TP_ENSINO',
    'TP_LOCALIZACAO_ESC',
    'TP_FAIXA_SALARIAL',
    
    # Questões socioeconômicas
    'Q001', 'Q002', 'Q005', 'Q006', 'Q025',
    
    # Infraestrutura
    'NU_INFRAESTRUTURA'
]

colunas_desempenho = [
    # Localização
    'SG_UF_PROVA',
    
    # Características do candidato
    'TP_SEXO',
    'TP_COR_RACA',
    'TP_DEPENDENCIA_ADM_ESC',
    'TP_ST_CONCLUSAO',
    'TP_FAIXA_ETARIA',
    
    # Notas
    'NU_NOTA_CN', 
    'NU_NOTA_CH', 
    'NU_NOTA_LC', 
    'NU_NOTA_MT',
    'NU_NOTA_REDACAO',
    
    # Categorias de desempenho
    'NU_DESEMPENHO',
    'TP_FAIXA_SALARIAL',

    # Questões socioeconômicas
    'Q001',
    'Q002',
    'Q005',
    'Q025'
]

In [3]:
# Carregar microdados completos
arquivo = '../Iniciação Científica/Códigos/microdados_tratado.parquet'

microdados = pd.read_parquet(arquivo)

dtypes = pd.read_json("dtypes.json", typ='series')
microdados = microdados.astype(dtypes)

microdados = microdados.sample(frac=0.1, random_state=42)  # Amostra de 10% dos dados

# Criar arquivos separados para cada aba
microdados[colunas_geral].to_parquet('sample_geral.parquet', index=False)
microdados[colunas_aspectos_sociais].to_parquet('sample_aspectos_sociais.parquet', index=False)
microdados[colunas_desempenho].to_parquet('sample_desempenho.parquet', index=False)

# Criar arquivos dtypes separados
dtypes_geral = {col: str(microdados[col].dtype) for col in colunas_geral if col in microdados.columns}
dtypes_aspectos = {col: str(microdados[col].dtype) for col in colunas_aspectos_sociais if col in microdados.columns}
dtypes_desempenho = {col: str(microdados[col].dtype) for col in colunas_desempenho if col in microdados.columns}

pd.Series(dtypes_geral).to_json('dtypes_geral.json')
pd.Series(dtypes_aspectos).to_json('dtypes_aspectos_sociais.json')
pd.Series(dtypes_desempenho).to_json('dtypes_desempenho.json')

KeyError: "['TP_ESCOLA'] not in index"

In [ ]:
# arquivo = '../Iniciação Científica/Códigos/microdados_tratado.parquet'

# microdados = pd.read_parquet(arquivo, engine='pyarrow',)

# dtypes = pd.read_json("dtypes.json", typ='series')
# microdados = microdados.astype(dtypes)

# microdados = microdados[colunas_desejadas]

# microdados = microdados[(microdados['TP_PRESENCA_GERAL'] == 1) & (microdados['NU_MEDIA_GERAL'] != -1)]

# microdados = microdados[microdados['NU_MEDIA_GERAL'] != -1]

# microdados = microdados.sample(938488, random_state=42)

# microdados.to_parquet('sample.parquet', index=False, engine='pyarrow')

In [ ]:
# df = pd.read_parquet('sample.parquet')
# f = df.select_dtypes(include='float16').columns
# i = df.select_dtypes(include='int16').columns

# df[f] = df[f].astype('float64')
# df[i] = df[i].astype('int64')
# profile = ProfileReport(df, title="Analise microdados")
# profile.to_file("exploratoria_original.html")